In [ ]:
import osmnx as ox
import networkx as nx
import folium
G=nx.Graph()

#座標を指定し地図データをダウンロード
G = ox.graph_from_point(center_point=(40.6156, -74.7706), network_type='drive', dist=3000)
# G = ox.graph_from_point(center_point=(38.5323, -77.0032), network_type='drive', dist=1000)
#グラフを可視化する
ox.plot_graph(G,node_color='#87cefa',save=True,filepath='kobe.jpg')

In [ ]:
node_ids = {i: n_id for i, n_id in enumerate(sorted(list(G.nodes)))}
i_node_ids = {n_id:i for i, n_id in enumerate(sorted(list(G.nodes)))}

In [ ]:
from vis import parse_input, parse_output, extract_partition

In [ ]:
ps50 = parse_output("output/map.in-1.in.out50")
ps300 = parse_output("output/map.in-1.in.out300")

In [ ]:
nid2pid50 = {}
for i, p in enumerate(ps50):
    for e in p[0]:
        nid2pid50[e] = i
        
nid2pid300 = {}
for i, p in enumerate(ps300):
    for e in p[0]:
        nid2pid300[e] = i

In [ ]:
len(G.nodes)

In [ ]:
for i in range(len(G.nodes)):
    print(i)
    for ps, nid2pid in zip([ps50, ps300], [nid2pid50, nid2pid300]):
        if (i not in nid2pid):
            continue
        elements_, path_, path_edges_ = extract_partition(ps, nid2pid[i])
        elements = [node_ids[e] for e in elements_]
        path = [node_ids[p] for p in path_]
        path_edges = [[p[0], p[1]] for p in path_edges_]

        node_color = []
        elements_set = set(elements)
        for n in G.nodes:
            if path[0] == n:
                node_color.append("g")
            elif path[-1] == n:
                node_color.append("y")
            elif n in elements_set:
                node_color.append("r")
            else:
                node_color.append("w")

        ox.plot_graph_routes(G, [path, [path[0]], [path[-1]]], node_color=node_color, 
                             route_colors=["b", "y", "g"], 
                             orig_dest_size=200, route_linewidths= [3.0, 0.0, 0.0], route_alpha=0.5)

In [ ]:
elements

In [ ]:
for i in range(70):
    if i not in nid2pid300:
        continue
    elements_, path_, path_edges_ = extract_partition(ps300, nid2pid300[i])
    elements = [node_ids[e] for e in elements_]
    path = [node_ids[p] for p in path_]
    path_edges = [[p[0], p[1]] for p in path_edges_]

    #ox.plot_graph(G,node_color='#87cefa')
    m = ox.plot_route_folium(G, path, tiles="openstreetmap")

    folium.Marker(popup="Start", location=(node_df[node_df.index == path[0]].y.values[0],
                            node_df[node_df.index == path[0]].x.values[0])).add_to(m)
    folium.Marker(icon=folium.Icon(icon="bell", color="green"), 
                  location=(node_df[node_df.index == path[-1]].y.values[0],
                            node_df[node_df.index == path[-1]].x.values[0])).add_to(m)

    for e in elements:
        folium.Marker(icon=folium.Icon(color='red', icon="eye-open"),
                      location=(node_df[node_df.index == e].y.values[0],
                            node_df[node_df.index == e].x.values[0])).add_to(m)

    m.save(f"map_{i}.html")